In [1]:
import pandas as pd
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
import configparser
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def init_config():
    config = configparser.ConfigParser()
    config.read("../config.ini")
    return config

def download_blob(config):
    """Downloads a blob from the bucket."""
    
    # Initialize a storage client
    path = config['gcp']['raw_data']
    df = pd.read_csv(path)
    return df

config = init_config()
df = download_blob(config)

In [3]:
categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    df[column] = df[column].str.replace(' ', '_').str.lower()

In [4]:
df.Department = df.Department.map(lambda x : "research_and_development" if x == 'research_&_development' else x) 

In [5]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,yes,travel_rarely,1102,sales,1,2,life_sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,no,travel_frequently,279,research_and_development,8,1,life_sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,yes,travel_rarely,1373,research_and_development,2,2,other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,no,travel_frequently,1392,research_and_development,3,4,life_sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,no,travel_rarely,591,research_and_development,2,1,medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [6]:
numerical_columns = df.select_dtypes(include=['int64'])

In [7]:
def create_and_replace_bins(df, columns, num_bins=5):
    binned_df = df.copy()
    for col in columns:
        col_range = df[col].max() - df[col].min()
        if col_range > 5:
            # Calculate quantile-based bins
            _, bins = pd.qcut(df[col], q=num_bins, duplicates='drop', retbins=True)
            # Adjust bins to integer values
            bins = np.floor(bins).astype(int)
            bins[-1] = bins[-1] + 1  # Ensure the last bin is inclusive
            # Create labels based on bin edges
            labels = [f'{bins[i]}<=x<{bins[i+1]}' for i in range(len(bins)-1)]
            # Apply binning and replace original column
            binned_df[col] = pd.cut(df[col], bins=bins, labels=labels, right=False, include_lowest=True)
    return binned_df

# Apply dynamic binning and replace columns with integer bins where range is > 5
df = create_and_replace_bins(df, numerical_columns, num_bins=5)

In [8]:
def get_binned_value_counts(df, columns):
    value_counts_dict = {}
    for col in columns:
        value_counts_dict[col] = df[col].value_counts().to_dict()
    return value_counts_dict

# Get value counts for each binned column
get_binned_value_counts(df, numerical_columns)


{'Age': {'29<=x<34': 316,
  '45<=x<61': 314,
  '38<=x<45': 308,
  '34<=x<38': 274,
  '18<=x<29': 258},
 'DailyRate': {'391<=x<656': 296,
  '1224<=x<1500': 296,
  '942<=x<1224': 294,
  '102<=x<391': 292,
  '656<=x<942': 292},
 'DistanceFromHome': {'2<=x<5': 359,
  '9<=x<17': 318,
  '17<=x<30': 297,
  '5<=x<9': 288,
  '1<=x<2': 208},
 'Education': {3: 572, 4: 398, 2: 282, 1: 170, 5: 48},
 'EmployeeCount': {1: 1470},
 'EmployeeNumber': {'1654<=x<2069': 295,
  '1<=x<402': 294,
  '402<=x<814': 294,
  '1235<=x<1654': 294,
  '814<=x<1235': 293},
 'EnvironmentSatisfaction': {3: 453, 4: 446, 2: 287, 1: 284},
 'HourlyRate': {'73<=x<87': 310,
  '45<=x<59': 301,
  '87<=x<101': 300,
  '30<=x<45': 282,
  '59<=x<73': 277},
 'JobInvolvement': {3: 868, 2: 375, 4: 144, 1: 83},
 'JobLevel': {1: 543, 2: 534, 3: 218, 4: 106, 5: 69},
 'JobSatisfaction': {4: 459, 3: 442, 1: 289, 2: 280},
 'MonthlyIncome': {'2695<=x<4228': 295,
  '5743<=x<9860': 295,
  '9860<=x<20000': 294,
  '1009<=x<2695': 293,
  '4228<=x<5

In [9]:
df['Attrition'] = df['Attrition'].map(lambda x : 0 if x == 'no' else 1) 


In [10]:
df.select_dtypes(include=['object'])

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
0,travel_rarely,sales,life_sciences,female,sales_executive,single,y,yes
1,travel_frequently,research_and_development,life_sciences,male,research_scientist,married,y,no
2,travel_rarely,research_and_development,other,male,laboratory_technician,single,y,yes
3,travel_frequently,research_and_development,life_sciences,female,research_scientist,married,y,yes
4,travel_rarely,research_and_development,medical,male,laboratory_technician,married,y,no
...,...,...,...,...,...,...,...,...
1465,travel_frequently,research_and_development,medical,male,laboratory_technician,married,y,no
1466,travel_rarely,research_and_development,medical,male,healthcare_representative,married,y,no
1467,travel_rarely,research_and_development,life_sciences,male,manufacturing_director,married,y,yes
1468,travel_frequently,sales,medical,male,sales_executive,married,y,no


In [11]:
#I would get rid of Marital Status and Gender -- whether or not keeping these featurs is based on the context 
#In this case you want to ensure that decisions are not influenced by gender or marital status, hence promoting fairness and ethics 
# you need to check potential correlation with other features and the target variable

# Encoding categorical variables
df['Gender'] = df['Gender'].map({'male': 1, 'female': 0})
df['MaritalStatus'] = df['MaritalStatus'].map({'single': 0, 'married': 1, 'divorced': 2})

# Contingency tables
contingency_table_gender = pd.crosstab(df['Gender'], df['Attrition'])
contingency_table_marital = pd.crosstab(df['MaritalStatus'], df['Attrition'])

print("Contingency Table - Gender vs Target:")
print(contingency_table_gender)

print("\nContingency Table - Marital Status vs Target:")
print(contingency_table_marital)

# Chi-Square tests
chi2_gender, p_gender, dof_gender, ex_gender = chi2_contingency(contingency_table_gender)
chi2_marital, p_marital, dof_marital, ex_marital = chi2_contingency(contingency_table_marital)

print(f"\nChi-Square Test between gender and target: chi2 = {chi2_gender}, p-value = {p_gender}")
print(f"Chi-Square Test between marital_status and target: chi2 = {chi2_marital}, p-value = {p_marital}")

# Interpretation
alpha = 0.05
print("\nInterpretation:")
if p_gender < alpha:
    print("There is a significant association between gender and target (p < 0.05).")
else:
    print("There is no significant association between gender and target (p >= 0.05).")

if p_marital < alpha:
    print("There is a significant association between marital status and target (p < 0.05).")
else:
    print("There is no significant association between marital status and target (p >= 0.05).")

Contingency Table - Gender vs Target:
Attrition    0    1
Gender             
0          501   87
1          732  150

Contingency Table - Marital Status vs Target:
Attrition        0    1
MaritalStatus          
0              350  120
1              589   84
2              294   33

Chi-Square Test between gender and target: chi2 = 1.1169671241970975, p-value = 0.29057244902890855
Chi-Square Test between marital_status and target: chi2 = 46.163676540848705, p-value = 9.45551106034083e-11

Interpretation:
There is no significant association between gender and target (p >= 0.05).
There is a significant association between marital status and target (p < 0.05).


In [12]:
df.drop(columns=['Gender','StandardHours', 'Over18', 'EmployeeCount', 'EmployeeNumber', 'BusinessTravel'], inplace=True)

In [13]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,38<=x<45,1,travel_rarely,942<=x<1224,sales,1<=x<2,2,life_sciences,2,87<=x<101,...,3,1,0,8<=x<10,0<=x<2,1,5<=x<7,4<=x<7,0<=x<1,4<=x<7
1,45<=x<61,0,travel_frequently,102<=x<391,research_and_development,5<=x<9,1,life_sciences,3,59<=x<73,...,4,4,1,10<=x<17,3<=x<4,3,10<=x<41,7<=x<19,1<=x<4,7<=x<18
2,34<=x<38,1,travel_rarely,1224<=x<1500,research_and_development,2<=x<5,2,other,4,87<=x<101,...,3,2,0,5<=x<8,3<=x<4,3,0<=x<2,0<=x<1,0<=x<1,0<=x<1
3,29<=x<34,0,travel_frequently,1224<=x<1500,research_and_development,2<=x<5,4,life_sciences,4,45<=x<59,...,3,3,0,8<=x<10,3<=x<4,3,7<=x<10,7<=x<19,1<=x<4,0<=x<1
4,18<=x<29,0,travel_rarely,391<=x<656,research_and_development,2<=x<5,1,medical,1,30<=x<45,...,3,4,1,5<=x<8,3<=x<4,3,2<=x<5,2<=x<4,1<=x<4,2<=x<4


In [14]:
# Business Travel

# df['BusinessTravel'].value_counts()
# # in this case we will use label encoding -- you should check the reason !! we could have choosen one-hot but we didn't ...
# df['BusinessTravel'] = df['BusinessTravel'].apply(lambda x: 0 if x =='Non-Travel' else (1 if x == 'Travel_Rarely' else 2))

# Department

df['Department'].value_counts()
department_dummies = pd.get_dummies(df['Department'], prefix='Department', dtype=float) # why now we used one-hot encoding and not label encoding?
df.drop(columns=['Department'], inplace=True)
df = pd.concat([df,department_dummies], axis=1)

# Education Field and JobRole

for field in ['JobRole', 'EducationField']:
    lb = preprocessing.LabelBinarizer()
    new_data = lb.fit_transform(df[field])
    binary_df = pd.DataFrame(new_data, columns=[f"{field}_{cls}" for cls in lb.classes_])
    df = pd.concat([df.drop(columns=[field]), binary_df], axis=1)


# OverTime

df['OverTime'] = df['OverTime'].map({'yes': 1, 'no': 0})

In [15]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_research_director,JobRole_research_scientist,JobRole_sales_executive,JobRole_sales_representative,EducationField_human_resources,EducationField_life_sciences,EducationField_marketing,EducationField_medical,EducationField_other,EducationField_technical_degree
0,38<=x<45,1,2,942<=x<1224,1<=x<2,2,2,87<=x<101,3,2,...,0,0,1,0,0,1,0,0,0,0
1,45<=x<61,0,2,102<=x<391,5<=x<9,1,3,59<=x<73,2,2,...,0,1,0,0,0,1,0,0,0,0
2,34<=x<38,1,2,1224<=x<1500,2<=x<5,2,4,87<=x<101,2,1,...,0,0,0,0,0,0,0,0,1,0
3,29<=x<34,0,2,1224<=x<1500,2<=x<5,4,4,45<=x<59,3,1,...,0,1,0,0,0,1,0,0,0,0
4,18<=x<29,0,2,391<=x<656,2<=x<5,1,1,30<=x<45,3,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
df.dtypes

In [17]:
from imblearn.over_sampling import SMOTENC

desired_minority_count = int(0.3 * 1233)
sampling_strategy = {0: 1233, 1: desired_minority_count}
X_train = df.drop(columns=['Attrition'])
y_train = df['Attrition']

smotenc = SMOTENC(categorical_features="auto", sampling_strategy=sampling_strategy, random_state=42)
X_train_resampled, y_train_resampled = smotenc.fit_resample(X_train, y_train)

# Combine resampled features with target
df = pd.concat([pd.DataFrame(X_train_resampled, columns=X_train.columns), pd.DataFrame(y_train_resampled, columns=['Attrition'])], axis=1)

In [18]:
df.columns

Index(['Age', 'BusinessTravel', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Department_human_resources',
       'Department_research_and_development', 'Department_sales',
       'JobRole_healthcare_representative', 'JobRole_human_resources',
       'JobRole_laboratory_technician', 'JobRole_manager',
       'JobRole_manufacturing_director', 'JobRole_research_director',
       'JobRole_research_scientist', 'JobRole_sales_executive',
       'JobRole_sales_representative', 'EducationField_human_resources',
       'EducationFiel

In [19]:
df.count()

Age                                    1602
BusinessTravel                         1602
DailyRate                              1602
DistanceFromHome                       1602
Education                              1602
EnvironmentSatisfaction                1602
HourlyRate                             1602
JobInvolvement                         1602
JobLevel                               1602
JobSatisfaction                        1602
MaritalStatus                          1602
MonthlyIncome                          1602
MonthlyRate                            1602
NumCompaniesWorked                     1602
OverTime                               1602
PercentSalaryHike                      1602
PerformanceRating                      1602
RelationshipSatisfaction               1602
StockOptionLevel                       1602
TotalWorkingYears                      1602
TrainingTimesLastYear                  1602
WorkLifeBalance                        1602
YearsAtCompany                  

In [20]:
df.Attrition.value_counts()

Attrition
0    1233
1     369
Name: count, dtype: int64

In [21]:
df.isnull().sum()

Age                                    0
BusinessTravel                         0
DailyRate                              0
DistanceFromHome                       0
Education                              0
EnvironmentSatisfaction                0
HourlyRate                             0
JobInvolvement                         0
JobLevel                               0
JobSatisfaction                        0
MaritalStatus                          0
MonthlyIncome                          0
MonthlyRate                            0
NumCompaniesWorked                     0
OverTime                               0
PercentSalaryHike                      0
PerformanceRating                      0
RelationshipSatisfaction               0
StockOptionLevel                       0
TotalWorkingYears                      0
TrainingTimesLastYear                  0
WorkLifeBalance                        0
YearsAtCompany                         0
YearsInCurrentRole                     0
YearsSinceLastPr

In [22]:
df.to_csv("gs://pa-poc-mlspec-3-cs/pre-processed4.csv", index=False)

In [23]:
def init_config():
    config = configparser.ConfigParser()
    config.read("../config.ini")
    return config

config = init_config()

In [24]:
# Will show you how to set up automl 

# First create a dataset 

import os

from google.cloud import aiplatform

PROJECT_ID = config['gcp']['project']
REGION = config['gcp']['region']

aiplatform.init(project=PROJECT_ID, location=REGION)

dataset = aiplatform.TabularDataset.create(
    display_name="HR Analytics3",
    gcs_source="gs://pa-poc-mlspec-3-cs/pre-processed4.csv",
)

label_column = "Attrition"

print(dataset.resource_name)
     

Creating TabularDataset
Create TabularDataset backing LRO: projects/928543623034/locations/europe-west2/datasets/1081409268336295936/operations/1493557353717432320
TabularDataset created. Resource name: projects/928543623034/locations/europe-west2/datasets/1081409268336295936
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/928543623034/locations/europe-west2/datasets/1081409268336295936')
projects/928543623034/locations/europe-west2/datasets/1081409268336295936


In [25]:
job = aiplatform.AutoMLTabularTrainingJob(
  display_name="train-automl-hr-analytics1",
  optimization_prediction_type="classification",
  optimization_objective="maximize-au-prc",
)

model = job.run(
    dataset=dataset,
    target_column=label_column,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    model_display_name="test1",
    disable_early_stopping=False,
)

No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.
The column transformation of type 'auto' was set for the following columns: ['JobRole_healthcare_representative', 'TrainingTimesLastYear', 'JobRole_research_scientist', 'NumCompaniesWorked', 'WorkLifeBalance', 'YearsInCurrentRole', 'RelationshipSatisfaction', 'HourlyRate', 'TotalWorkingYears', 'EducationField_other', 'EducationField_life_sciences', 'JobRole_manufacturing_director', 'JobRole_sales_representative', 'DailyRate', 'YearsWithCurrManager', 'EducationField_medical', 'PercentSalaryHike', 'EducationField_human_resources', 'Age', 'Education', 'JobLevel', 'EducationField_technical_degree', 'JobInvolvement', 'JobRole_manager', 'MaritalStatus', 'Department_research_and_development', 'JobRole_human_resources', 'EducationField_marketing', 'YearsSinceLastPromotion', 'YearsAtCompany', 'EnvironmentSatisfaction', 'Department_human_resources', 'JobRole_laboratory_techn

In [26]:
endpoint = model.deploy(deployed_model_display_name='test1',
    machine_type='n1-standard-4'
)

Creating Endpoint
Create Endpoint backing LRO: projects/928543623034/locations/europe-west2/endpoints/7600004691094667264/operations/9211319740146909184
Endpoint created. Resource name: projects/928543623034/locations/europe-west2/endpoints/7600004691094667264
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/928543623034/locations/europe-west2/endpoints/7600004691094667264')
Deploying model to Endpoint : projects/928543623034/locations/europe-west2/endpoints/7600004691094667264
Deploy Endpoint model backing LRO: projects/928543623034/locations/europe-west2/endpoints/7600004691094667264/operations/4680698515012190208
Endpoint model deployed. Resource name: projects/928543623034/locations/europe-west2/endpoints/7600004691094667264
